# 🛒 **연관 분석 (Association Rule Analysis) 과제**

### <span style="color:black; background-color:#F5F5F5;"> **Q1. 연관 규칙 {우유} → {쿠키}가 도출되었을 때, 다음 용어들이 각각 무엇을 의미하는지 설명하시오.**
#### <span style="color:black; background-color:#F5F5F5;">**① 지지도(support) ② 신뢰도(confidence) ③ 향상도(lift)** </span>

답:

1. 지지도 - 전체 거래 항목 중 우유와 쿠키가 동시에 포함되는 거래의 비율

2. 신뢰도 - 우유를 포함하는 거래들 중 우유와 쿠키를 동시에 포함하는 거래의 비율

3. 향상도 - 우유가 주어지지 않았을 때의 쿠키의 거래 확률 대비, 우유가 주어졌을 때의 쿠키의 거래 확률의 증가 비율

### <span style="color:black; background-color:#F5F5F5;"> **Q2. Apriori 알고리즘이 처리해야 할 후보 항목 수가 기하급수적으로 증가하는 이유와, FP-Growth가 이를 어떻게 해결하는지 설명하시오.**

답:

빈발 항목 집합에 대해 단계적으로 조합 생성 및 필터링하는 과정을 처리하는데, 각 단계에서 데이터를 다시 스캔해야하기 때문에 처리해야하는 후보 항목 수가 기하급수적으로 증가한다.

FP-Growth 알고리즘은 FP-Tree의 구조(트리, 배열, 연결리스트의 합친 구조)를 이용하여 데이터를 2번만 스캔하기 때문에, 연산 속도를 높여서 해결하였다.

# <span style="color:black; background-color:#F5F5F5;"> 💸 **연관 분석을 활용한 잉마트(Ing-Mart) 고객 장바구니 패턴 분석 및 비즈니스 전략 수립** </span>

<strong>죽지도 않고 다시 돌아온 잉마트..! 🤣🫥😫🙃<br>
이번 연관 분석 심화 세션의 과제는 잉마트의 고객 장바구니 분석과 전략 수립입니다~ <strong>



<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">
<strong> 🤓 지난 심화 세션에서 배운 개념과 실습 내용을 바탕으로 아래 빈칸을 채워주시고, 해당 장바구니 결과를 분석하여 이에 적합한 전략을 제시해주시면 됩니다! <strong>
</span>

## **1️⃣ 데이터 불러오기 및 전처리**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
%pip install mlxtend

- 데이터 불러오기
  - 1차 인사이콘 때 활용하셨던 데이터 원본을 활용해주시면 됩니다!
  - 알맞게 경로 지정해주세요~

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
transaction_data = pd.read_csv('/content/drive/MyDrive/💡INSIGHT/transaction_data.csv')
product = pd.read_csv('/content/drive/MyDrive/💡INSIGHT/product.csv')

In [7]:
transaction_data.head()

,Household_ID,Basket_ID,Product_ID,Store_ID,Day,Quantity,Sales_Value,Trans_time,Week_no,Disc(retail),Disc(coupon),Disc(coupon_match)
0,1803,30780785930,1065887,338,252,1,8.99,1419,37,-1.00,0.0,0.0
1,2299,33768622588,1073244,446,456,1,1.00,2030,66,-1.59,0.0,0.0
2,158,30202616809,7025114,343,225,1,1.50,1246,33,-0.50,-1.0,0.0
3,2347,42076926172,1064299,438,695,1,2.50,1430,100,-2.89,0.0,0.0
4,1430,31625201009,1040197,31742,312,1,2.29,1423,45,0.00,0.0,0.0


- 이상치 처리
  - 예시로 제시해드리지만 추가적으로 필요한 부분은 전처리 처리해주세요!

In [8]:
# Quantity 열에 대해 Z-score를 계산한 뒤, 절댓값을 취해 새로운 열 'z_score'에 저장
transaction_data["z_score"] = np.abs(stats.zscore(transaction_data["Quantity"]))

# Z-score가 3을 초과하는 이상치(즉, 평균에서 3표준편차 이상 벗어난 값)를 추출
outliers_zscore = transaction_data[transaction_data["z_score"] > 3]

- 고객 - 상품 행렬 생성

In [9]:
# 고객-상품 pivot_table 생성 (행: 고객, 열: 상품, 값: 총 구매금액)
user_item_matrix = transaction_data.pivot_table(
    index='Household_ID',     # 가구 ID 기준
    columns='Product_ID',     # 상품 ID 기준
    values='Sales_Value',     # 구매 금액
    aggfunc='sum',            # 상품별 총 구매금액
    fill_value=0              # 구매 이력 없으면 0
)
user_item_matrix

Product_ID,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,18273019,18273051,18273115,18273133,18292005,18293142,18293439,18293696,18294080,18316298
Household_ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 행렬의 크기 확인 (고객 수 × 상품 수)
user_item_matrix.shape

(2500, 92339)

- 구매가 적은 사용자/상품 필터링

In [11]:
# 필터링 기준 정의
min_product_purchases = 10   # 최소 10명 이상이 구매한 상품만 사용
min_user_purchases = 2       # 최소 2개 이상 상품 구매한 사용자만 사용

# 상품별 구매된 고객 수 계산
product_purchase_count = (user_item_matrix > 0).sum()

# 고객별 구매한 상품 수 계산
user_purchase_count = (user_item_matrix > 0).sum(axis=1)

# 기준을 만족하는 상품과 사용자 필터링
filtered_products = product_purchase_count[product_purchase_count >= min_product_purchases].index
filtered_users = user_purchase_count[user_purchase_count >= min_user_purchases].index

# 필터링된 행렬 추출
filtered_matrix = user_item_matrix.loc[filtered_users, filtered_products]
print(f"\n2. Filtered Matrix Shape: {filtered_matrix.shape}")


2. Filtered Matrix Shape: (2500, 23326)


- 이상치 및 음수 제거한 트랜잭션 데이터 생성

In [12]:
# Z-score 기준으로 이상치 제거 (±3 이상) + 구매 수량이 양수인 데이터만 남김
transaction_data_cleaned = transaction_data[
    (transaction_data['z_score'] < 3) &
    (transaction_data['z_score'] > -3) &
    (transaction_data['Quantity'] > 0)
]
print(transaction_data_cleaned.shape)

# 데이터 샘플 확인
transaction_data_cleaned.head()

(2559321, 13)


,Household_ID,Basket_ID,Product_ID,Store_ID,Day,Quantity,Sales_Value,Trans_time,Week_no,Disc(retail),Disc(coupon),Disc(coupon_match),z_score
0,1803,30780785930,1065887,338,252,1,8.99,1419,37,-1.00,0.0,0.0,0.086202
1,2299,33768622588,1073244,446,456,1,1.00,2030,66,-1.59,0.0,0.0,0.086202
2,158,30202616809,7025114,343,225,1,1.50,1246,33,-0.50,-1.0,0.0,0.086202
3,2347,42076926172,1064299,438,695,1,2.50,1430,100,-2.89,0.0,0.0,0.086202
4,1430,31625201009,1040197,31742,312,1,2.29,1423,45,0.00,0.0,0.0,0.086202


- 정제된 데이터로 다시 사용자-상품 행렬 생성 및 필터링

In [13]:
# 더 정확한 연관 규칙 도출을 위해 이상치 제거 후 재생성
# 정제된 데이터를 기반으로 사용자-상품 매트릭스 다시 생성
user_item_matrix = transaction_data_cleaned.pivot_table(
    index='Household_ID',     # 가구 ID 기준
    columns='Product_ID',     # 상품 ID 기준
    values='Sales_Value',     # 구매 금액
    aggfunc='sum',            # 상품별 총 구매금액
    fill_value=0              # 구매 이력 없으면 0
)

# 필터링 기준 재사용
min_product_purchases = 10
min_user_purchases = 2

# 상품/사용자별 구매 횟수 계산
product_purchase_count = (user_item_matrix > 0).sum()
user_purchase_count = (user_item_matrix > 0).sum(axis=1)

# 조건에 맞는 상품과 사용자 필터
filtered_products = product_purchase_count[product_purchase_count >= min_product_purchases].index
filtered_users = user_purchase_count[user_purchase_count >= min_user_purchases].index

# 최종 필터링된 행렬 생성
filtered_matrix = user_item_matrix.loc[filtered_users, filtered_products]
filtered_matrix

Product_ID,27658,34873,43020,43871,59666,138619,197681,201704,215923,244960,...,18005913,18005929,18022252,18055205,18055329,18105264,18106286,18119016,18147612,18203921
Household_ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 상품 정보 조인하여 제품 타입 단위로 분석 준비

In [14]:
# Product_ID을 기준으로 데이터 product와 inner join하여 Product_type 정보 추가
merged_data = pd.merge(transaction_data_cleaned, product, on='Product_ID', how='inner')
merged_data

,Household_ID,Basket_ID,Product_ID,Store_ID,Day,Quantity,Sales_Value,Trans_time,Week_no,Disc(retail),Disc(coupon),Disc(coupon_match),z_score,Manufacturer,Brand,Category,Subcategory,Product_type,Curr_Size_of_Product
0,1803,30780785930,1065887,338,252,1,8.99,1419,37,-1.00,0.0,0.0,0.086202,69,Private,DRUG GM,ANALGESICS,ADULT ANALGESICS,
1,2299,33768622588,1073244,446,456,1,1.00,2030,66,-1.59,0.0,0.0,0.086202,2110,National,GROCERY,ICE CREAM/MILK/SHERBTS,PREMIUM PINTS,PT
2,158,30202616809,7025114,343,225,1,1.50,1246,33,-0.50,-1.0,0.0,0.086202,1838,National,GROCERY,BAKED BREAD/BUNS/ROLLS,MAINSTREAM WHEAT/MULTIGRAIN BR,20 OZ
3,2347,42076926172,1064299,438,695,1,2.50,1430,100,-2.89,0.0,0.0,0.086202,2193,National,GROCERY,ICE CREAM/MILK/SHERBTS,PREMIUM,48 OZ
4,1430,31625201009,1040197,31742,312,1,2.29,1423,45,0.00,0.0,0.0,0.086202,869,National,GROCERY,TEAS,TEA BAGS HERBAL & FLAVORED,20 CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559316,225,33655617762,6463742,31642,447,1,2.29,1942,65,-1.70,0.0,0.0,0.086202,69,Private,GROCERY,CHEESE,SHREDDED CHEESE,16 OZ
2559317,266,40097402794,968269,367,551,1,1.99,2255,79,0.00,0.0,0.0,0.086202,1091,National,GROCERY,HOUSEHOLD CLEANG NEEDS,TOILET BOWL MANUAL,24 OZ
2559318,2364,30443976140,2041688,673,230,2,1.50,1757,34,-0.62,0.0,0.0,0.085335,69,Private,GROCERY,MEAT - SHELF STABLE,CHILI: CANNED,15 OZ
2559319,1048,31145396229,1082185,436,280,1,0.74,1445,41,0.00,0.0,0.0,0.086202,2,National,PRODUCE,TROPICAL FRUIT,BANANAS,40 LB


- 장바구니(Basket_ID)별로 구매한 상품타입 목록 정리

In [15]:
# 각 거래(Basket_ID)마다 구매한 상품 유형(Product_type)의 리스트 생성
transactions = merged_data.groupby('Basket_ID')['Product_type'].unique().reset_index()
transactions

,Basket_ID,Product_type
0,26984851472,"[POTATOES RUSSET (BULK&BAG), CELERY, ONIONS SW..."
1,26984851516,"[HAMBURGER BUNS, TRAY PACK/CHOC CHIP COOKIES, ..."
2,26984896261,"[GRANOLA BARS, EGGS - X-LARGE, LINKS - RAW, SN..."
3,26984905972,"[RAMEN NOODLES/RAMEN CUPS, MAINSTREAM WHITE BR..."
4,26984945254,"[SEASONAL CANDY BOX NON-CHOCOLA, INSIDE FROST ..."
...,...,...
254725,42302712006,"[TORTILLA/NACHO CHIPS, SFT DRNK 2 LITER BTL CA..."
254726,42302712189,"[PLSTC CTLRYTBLCLTHSTTHPKSST, REFRIG DIPS, PAP..."
254727,42302712298,"[BUTTER, HAIR CONDITIONERS AND RINSES, SHAMPOO..."
254728,42305362497,"[SEASONAL MISCELLANEOUS, SFT DRNK 2 LITER BTL ..."


- 트랜잭션 리스트로 변환

In [16]:
transaction_list = transactions['Product_type'].tolist()
transaction_list = [list(item) for item in transaction_list]
print(transaction_list[:5])

[['POTATOES RUSSET (BULK&BAG)', 'CELERY', 'ONIONS SWEET (BULK&BAG)', 'ORGANIC CARROTS', 'BANANAS'], ['HAMBURGER BUNS', 'TRAY PACK/CHOC CHIP COOKIES', 'PEANUT BUTTER', 'SPONGES: BATH HOUSEHOLD', 'GRAHAM CRACKERS'], ['GRANOLA BARS', 'EGGS - X-LARGE', 'LINKS - RAW', 'SNACK CRACKERS', 'GRND/PATTY - ROUND'], ['RAMEN NOODLES/RAMEN CUPS', 'MAINSTREAM WHITE BREAD'], ['SEASONAL CANDY BOX NON-CHOCOLA', 'INSIDE FROST BULBS', 'CHEWING GUM']]


- 트랜잭션 통계

In [17]:
transactions['num_products'] = transactions['Product_type'].apply(len)
average_products_per_order = transactions['num_products'].mean()
max_products_per_order = transactions['num_products'].max()
min_products_per_order = transactions['num_products'].min()

print(f"Average number of products per order: {average_products_per_order}")
print(f"Maximum number of products per order: {max_products_per_order}")
print(f"Minimum number of products per order: {min_products_per_order}")

Average number of products per order: 8.520123267773721
Maximum number of products per order: 128
Minimum number of products per order: 1


<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">
<strong> 🤓 지금부턴 지난 실습 때 했던 과정의 반복! <strong>
</span>

## **2️⃣ 연관 분석 - TransactionEncoder로 이진 행렬로 변환**

In [18]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transaction_list).transform(transaction_list)  # 학습과 변환을 따로따로!

df_encoded = pd.DataFrame(te_ary, columns=te.columns_).astype(int)
df_encoded.head()

,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,20 BIKES,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **3️⃣ 연관 분석 - Apriori 알고리즘 활용**

- Apriori 알고리즘으로 빈발 항목 집합 도출 (지지도 0.05% 이상)

In [19]:
from mlxtend.frequent_patterns import apriori

filtered_onehot = df_encoded.loc[:, df_encoded.sum(axis=0) > 20]

# apriori로 frequent_itemsets 추출 (최소지지도는 0.005, use_colnames=True, low_memory=True)
frequent_itemsets = apriori(filtered_onehot, min_support=0.005, use_colnames=True, low_memory=True)
frequent_itemsets.head()

,support,itemsets
0,0.011899,(ADULT ANALGESICS)
1,0.030020,(ADULT CEREAL)
2,0.005374,(AIR CARE - CONTINUOUS ACTION)
3,0.008024,(ALKALINE BATTERIES)
4,0.040266,(ALL FAMILY CEREAL)


- `association_rules`로 연관 규칙 도출 및 필터링(신뢰도 40% 이상)

In [23]:
num_itemsets = len(frequent_itemsets)

# confidence 기준으로 연관 규칙(rules)을 추출하세요.
# 조건: min_threshold=0.4, metric="***",num_itemsets = num_itemsets
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4, num_itemsets = num_itemsets)

rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head()

,antecedents,consequents,support,confidence,lift
0,(ADULT CEREAL),(BANANAS),0.012119,0.403688,3.393439
1,(ADULT CEREAL),(FLUID MILK WHITE ONLY),0.019154,0.638028,2.650870
2,(ALL FAMILY CEREAL),(FLUID MILK WHITE ONLY),0.026137,0.649118,2.696946
3,(APPLE JUICE & CIDER (OVER 50%),(FLUID MILK WHITE ONLY),0.006874,0.588769,2.446211
4,(APPLES GALA (BULK&BAG)),(BANANAS),0.006996,0.507837,4.268928


In [24]:
print(rules.shape)

(445, 14)


- 연관 분석용 리스트 구조 정리

In [25]:
transaction_list = [list(t) if isinstance(t, (list, np.ndarray)) else [t] for t in transaction_list]
transaction_list = [t.tolist() if isinstance(t, np.ndarray) else list(t) if isinstance(t, list) else [t] for t in transaction_list]

- 불필요한 지표 제거

In [26]:
apriori = rules.drop(columns=[
    "antecedent support",
    "consequent support",
    "representativity",
    "conviction",
    "zhangs_metric",
    "jaccard",
    "certainty",
    "kulczynski"
])

- 유의미한 규칙 필터링(향상도 1 이상)

In [27]:
apriori = apriori[apriori['lift'] >= 1]

- 결과 확인

In [28]:
apriori

,antecedents,consequents,support,confidence,lift,leverage
0,(ADULT CEREAL),(BANANAS),0.012119,0.403688,3.393439,0.008547
1,(ADULT CEREAL),(FLUID MILK WHITE ONLY),0.019154,0.638028,2.650870,0.011928
2,(ALL FAMILY CEREAL),(FLUID MILK WHITE ONLY),0.026137,0.649118,2.696946,0.016446
3,(APPLE JUICE & CIDER (OVER 50%),(FLUID MILK WHITE ONLY),0.006874,0.588769,2.446211,0.004064
4,(APPLES GALA (BULK&BAG)),(BANANAS),0.006996,0.507837,4.268928,0.005357
...,...,...,...,...,...,...
440,"(SHREDDED CHEESE, KIDS CEREAL)",(MAINSTREAM WHITE BREAD),0.005076,0.442202,4.200561,0.003868
441,"(SOFT DRINKS 12/18&15PK CAN CAR, KIDS CEREAL)",(MAINSTREAM WHITE BREAD),0.005135,0.454798,4.320211,0.003946
442,"(POTATO CHIPS, SHREDDED CHEESE)",(MAINSTREAM WHITE BREAD),0.005602,0.405974,3.856424,0.004149
443,"(SNACK CAKE - MULTI PACK, SOFT DRINKS 12/18&15...",(MAINSTREAM WHITE BREAD),0.005429,0.478878,4.548949,0.004236


## **[ 참고 ] 연관 분석 - FP-Growth 알고리즘 활용**

- 코드를 돌릴 때 조심해주세요!

In [29]:
filtered_onehot

,ABRASIVES,ACNE MEDICATIONS,ACTIVITY,ADDITIVES/FLUIDS,ADHESIVES/CAULK,ADULT ANALGESICS,ADULT CEREAL,ADULT INCONTINENCE BRIEFS,ADULT INCONTINENCE MISC PRODUC,ADULT INCONTINENCE PADS,...,WRAP,WREATHS,WREATHS/TINSEL/GARLAND,WRITING INSTRUMENTS,XMAS PLUSH,YEAST: DRY,YELLOW SUMMER SQUASH,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254726,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">
<strong> 🥹 실습 때 다루었던 FP-Growth 알고리즘이 참고 자료가 된 이유는 다음 셀 때문이에요... 돌릴 때 조심해주세요... 30분씩 걸릴 때도 있거든요... <strong>
</span>

<span style="color:black; background-color:#fff5b1; padding:2px 4px; border-radius:4px">
<strong> 🤔 "엥? 근데 FP-Growth 알고리즘이 Apriori 알고리즘보다 계산이 빠르다고 하지 않았나?"<strong>
</span>

<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">
<strong> 🤓 네! 공부를 열심히 하셨군요!? 맞습니다! 이론상 FP-Growth 알고리즘이 Apriori 알고리즘이 계산이 더 빠릅니다!<strong>
</span>

<span style="color:black; background-color:#fff5b1; padding:2px 4px; border-radius:4px">
<strong> 🤔 엥 그럼 왜...? <strong>
</span>

<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">
<strong> 🤓 사용한 라이브러리의 차이입니다! 저희는 mlxtend 라이브러리를 사용했습니다! <strong>
</span>

<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">
<strong> 🤓 mlxtend 라이브러리의 경우 Apriori는 Cython으로 최적화 되어 매우 빠르게 작동하지만, FP-Growth는 순수 Python으로 구현하기 때문에 오히려 느릴 수 있습니다. 이 경우 fpgrowth_py 라이브러리를 활용한다면 더 빠르게 작동할 수 있어요~ <strong>
</span>

- FP-Growth 알고리즘으로 빈발 항목 집합 도출 (지지도 0.05% 이상)

In [30]:
from mlxtend.frequent_patterns import fpgrowth

# FP-Growth 알고리즘을 사용해 frequent_itemsets_fp을 생성하세요.
# 조건: 최소지지도: 0.005, use_colnames=True, 입력 데이터는 boolean 타입(astype(bool))
frequent_itemsets_fp = fpgrowth(filtered_onehot.astype(bool), min_support=0.005, use_colnames=True)
frequent_itemsets_fp.head()

,support,itemsets
0,0.118961,(BANANAS)
1,0.038044,(POTATOES RUSSET (BULK&BAG))
2,0.023715,(ONIONS SWEET (BULK&BAG))
3,0.018816,(CELERY)
4,0.040133,(HAMBURGER BUNS)


- `association_rules`로 연관 규칙 도출 및 필터링(신뢰도 40% 이상)

In [31]:
# confidence 기준으로 연관 규칙(rules)을 추출하세요.
# 조건: min_threshold=0.4, metric="***"
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.4)

rules_fp.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(BANANAS),(FLUID MILK WHITE ONLY),0.118961,0.240686,0.061402,0.516154,2.144508,1.0,0.032770,1.569328,0.605754,0.205878,0.362785,0.385633
1,(POTATOES RUSSET (BULK&BAG)),(FLUID MILK WHITE ONLY),0.038044,0.240686,0.019236,0.505624,2.100759,1.0,0.010079,1.535903,0.544704,0.074129,0.348917,0.292773
2,"(POTATOES RUSSET (BULK&BAG), BANANAS)",(FLUID MILK WHITE ONLY),0.010788,0.240686,0.006776,0.628093,2.609593,1.0,0.004179,2.041677,0.623525,0.027690,0.510207,0.328123
3,"(POTATOES RUSSET (BULK&BAG), MAINSTREAM WHITE ...",(FLUID MILK WHITE ONLY),0.011463,0.240686,0.007577,0.660959,2.746144,1.0,0.004818,2.239592,0.643226,0.030979,0.553490,0.346219
4,"(POTATOES RUSSET (BULK&BAG), SHREDDED CHEESE)",(FLUID MILK WHITE ONLY),0.008519,0.240686,0.005606,0.658065,2.734118,1.0,0.003556,2.220635,0.639701,0.023013,0.549678,0.340678


In [32]:
from IPython.display import display

print("🔥 Frequent Itemsets:")
display(frequent_itemsets_fp)

print("\n🔥 Association Rules:")
display(rules_fp)

🔥 Frequent Itemsets:


,support,itemsets
0,0.118961,(BANANAS)
1,0.038044,(POTATOES RUSSET (BULK&BAG))
2,0.023715,(ONIONS SWEET (BULK&BAG))
3,0.018816,(CELERY)
4,0.040133,(HAMBURGER BUNS)
...,...,...
1475,0.005221,"(PAPER NAPKINS, FLUID MILK WHITE ONLY)"
1476,0.007267,"(FLUID MILK WHITE ONLY, STRING CHEESE)"
1477,0.006866,"(FLUID MILK WHITE ONLY, CANDY BARS (MULTI PACK))"
1478,0.006572,"(FLUID MILK WHITE ONLY, ISOTONIC DRINKS SINGLE..."



🔥 Association Rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(BANANAS),(FLUID MILK WHITE ONLY),0.118961,0.240686,0.061402,0.516154,2.144508,1.0,0.032770,1.569328,0.605754,0.205878,0.362785,0.385633
1,(POTATOES RUSSET (BULK&BAG)),(FLUID MILK WHITE ONLY),0.038044,0.240686,0.019236,0.505624,2.100759,1.0,0.010079,1.535903,0.544704,0.074129,0.348917,0.292773
2,"(POTATOES RUSSET (BULK&BAG), BANANAS)",(FLUID MILK WHITE ONLY),0.010788,0.240686,0.006776,0.628093,2.609593,1.0,0.004179,2.041677,0.623525,0.027690,0.510207,0.328123
3,"(POTATOES RUSSET (BULK&BAG), MAINSTREAM WHITE ...",(FLUID MILK WHITE ONLY),0.011463,0.240686,0.007577,0.660959,2.746144,1.0,0.004818,2.239592,0.643226,0.030979,0.553490,0.346219
4,"(POTATOES RUSSET (BULK&BAG), SHREDDED CHEESE)",(FLUID MILK WHITE ONLY),0.008519,0.240686,0.005606,0.658065,2.734118,1.0,0.003556,2.220635,0.639701,0.023013,0.549678,0.340678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,(FACIAL TISSUE & PAPER HANDKE),(FLUID MILK WHITE ONLY),0.018141,0.240686,0.008919,0.491668,2.042778,1.0,0.004553,1.493737,0.519902,0.035690,0.330538,0.264363
441,(PAPER NAPKINS),(FLUID MILK WHITE ONLY),0.010717,0.240686,0.005221,0.487179,2.024127,1.0,0.002642,1.480662,0.511441,0.021209,0.324626,0.254436
442,(STRING CHEESE),(FLUID MILK WHITE ONLY),0.014133,0.240686,0.007267,0.514167,2.136253,1.0,0.003865,1.562910,0.539515,0.029353,0.360168,0.272179
443,(ISOTONIC DRINKS SINGLE SERVE),(FLUID MILK WHITE ONLY),0.015762,0.240686,0.006572,0.416936,1.732282,1.0,0.002778,1.302283,0.429497,0.026300,0.232118,0.222120


In [33]:
fp_growth = rules_fp.drop(columns=[
    "antecedent support",
    "consequent support",
    "representativity",
    "conviction",
    "zhangs_metric",
    "jaccard",
    "certainty",
    "kulczynski"
])

- 유의미한 규칙 필터링(향상도 1 이상)

In [34]:
fp_growth = fp_growth[fp_growth['lift'] >= 1]
fp_growth

,antecedents,consequents,support,confidence,lift,leverage
0,(BANANAS),(FLUID MILK WHITE ONLY),0.061402,0.516154,2.144508,0.032770
1,(POTATOES RUSSET (BULK&BAG)),(FLUID MILK WHITE ONLY),0.019236,0.505624,2.100759,0.010079
2,"(POTATOES RUSSET (BULK&BAG), BANANAS)",(FLUID MILK WHITE ONLY),0.006776,0.628093,2.609593,0.004179
3,"(POTATOES RUSSET (BULK&BAG), MAINSTREAM WHITE ...",(FLUID MILK WHITE ONLY),0.007577,0.660959,2.746144,0.004818
4,"(POTATOES RUSSET (BULK&BAG), SHREDDED CHEESE)",(FLUID MILK WHITE ONLY),0.005606,0.658065,2.734118,0.003556
...,...,...,...,...,...,...
440,(FACIAL TISSUE & PAPER HANDKE),(FLUID MILK WHITE ONLY),0.008919,0.491668,2.042778,0.004553
441,(PAPER NAPKINS),(FLUID MILK WHITE ONLY),0.005221,0.487179,2.024127,0.002642
442,(STRING CHEESE),(FLUID MILK WHITE ONLY),0.007267,0.514167,2.136253,0.003865
443,(ISOTONIC DRINKS SINGLE SERVE),(FLUID MILK WHITE ONLY),0.006572,0.416936,1.732282,0.002778


## **4️⃣ 연관 분석 - 결과 해석**

<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">
<strong> 🤓 결과를 해석하고 전략을 세우는 게 해당 과제의 핵심이니 꼭!!!! 성의있게 깊게 고민한 흔적을 남겨주세요! <strong>
</span>

### **[연관분석] 지지도 0.9% 이상, 신뢰도 55% 이상, 향상도 1 이상 연관 분석**

In [36]:
results = apriori[(apriori['confidence']>0.55)&(apriori['support']>0.009)&(apriori['lift']>1)]
results

,antecedents,consequents,support,confidence,lift,leverage
1,(ADULT CEREAL),(FLUID MILK WHITE ONLY),0.019154,0.638028,2.650870,0.011928
2,(ALL FAMILY CEREAL),(FLUID MILK WHITE ONLY),0.026137,0.649118,2.696946,0.016446
48,(CHOCOLATE MILK),(FLUID MILK WHITE ONLY),0.021615,0.654308,2.718510,0.013664
51,(CORN),(FLUID MILK WHITE ONLY),0.013379,0.555682,2.308742,0.007584
57,(DAIRY CASE 100% PURE JUICE - O),(FLUID MILK WHITE ONLY),0.037451,0.588163,2.443691,0.022126
62,(EGGS - LARGE),(FLUID MILK WHITE ONLY),0.029667,0.571547,2.374658,0.017174
64,(EGGS - X-LARGE),(FLUID MILK WHITE ONLY),0.021752,0.583263,2.423334,0.012776
70,(FRUIT BOWL AND CUPS),(FLUID MILK WHITE ONLY),0.011710,0.572773,2.379748,0.006790
94,(KIDS CEREAL),(FLUID MILK WHITE ONLY),0.033000,0.654316,2.718544,0.020861
100,(MACARONI & CHEESE DNRS),(FLUID MILK WHITE ONLY),0.017100,0.572706,2.379471,0.009914


- 위 결과를 지지도, 신뢰도, 향상도 값을 바탕으로 해석해주세요! (두 가지 이상)

<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">  
<strong> 🤓 [ 예시 ] 'FLUID MILK WHITE ONLY'(우유)는 다양한 품목과 높은 결합 구매 패턴을 보이며, 30개 이상의 제품과 lift 2 이상으로 강한 연관성을 나타낸다. <br> 특히 시리얼 계열인 'ALL FAMILY CEREAL', 'KIDS CEREAL', 'ADULT CEREAL'은 각각 65% 이상의 신뢰도와 2.6~2.7의 lift를 기록해 눈에 띄는 결합 소비가 확인된다. <br> 전반적으로 우유는 시리얼, 과일, 베이커리, 아침 식사류 제품들과 자주 함께 구매되며, 이는 소비자의 식사 준비 맥락과 밀접하게 연결된 구매 경향을 보여준다. <strong>  
</span>

- 해석 1:

전반적으로 'MAINSTREAM WHITE BREAD'를 포함한 상품의 평균 신뢰도는 약 65%이고, 평균 향상도는 약 2.73이기 때문에, 이 또한 우유 상품에 대해 유의미한 결합 소비가 되는 품목인 것이 확인된다. 소비자들이 빵과 우유를 같이 구매하는 경우가 자주 있다는 것이다.

- 해석 2:

'CHOCOLATE MILK' 품목의 지지도는 약 2%, 신뢰도는 약 65%, 향상도는 약 2.72이다. 전반적으로 흰 우유 상품과 초코 우유 상품이 함께 구매하는 경우가 많다는 것을 확인하였다. 아마 높은 확률로 두 상품이 같이 진열되어있을 것이고, 같은 기업의 제품일 것이기 때문에 소비자들이 같이 구매하는 경우가 많은 것 같다.

## **5️⃣ 연관 분석 - 비즈니스 전략 수립**

- 위 결과해석에 따라 비즈니스 전략을 수립해주세요! (2가지 이상) -> 냅다 GPT만 패서 쓰지 말아주세요 . . .. .

<span style="color:black; background-color:#E6E6FA; padding:2px 4px; border-radius:4px">  
<strong> 🤓 [ 예시 ] 우유와 함께하는 식탁 큐레이션 존 구성 <strong>  
</span>

**🎯 목적**

* 우유와 자주 함께 구매되는 상품을 모아 **직관적인 구매 유도**
* **객단가 상승**, **편리한 쇼핑 경험 제공**

**🛒 구성 품목**

* **시리얼류**: KIDS / ALL FAMILY / ADULT CEREAL
* **베이커리**: 식빵, 비스킷, 토스터 페이스트리
* **과일류**: 바나나, 컵과일
* **유제품/간편식**: 요거트, 달걀, 마카로니 등

**📍 운영 방법**

* 우유 냉장고 인근에 **"우유와 최고의 궁합!"** 테마존 설치
* POP/QR코드로 **추천 식단**이나 **할인 쿠폰** 제공
* 계절별 테마 구성 (예: 여름=냉과일, 겨울=오트밀)

**📈 기대 효과**

* 우유 결합 구매율 상승
* 연관 제품 매출 증가
* ‘고민 없는 조합’으로 **고객 만족도 향상**



- 전략 1: 

### [🍼우유에 빠져봐유~🥛]

우유 상품 다양화를 통한 우유 품목 판매량 증가 

🐮 목적 : 

- 우유 상품들의 종류를 확대하여 소비자들의 더 다양한 우유 상품 구매 유도

🥣 구성 품목 :

- CHOCOLATE MILK, BANANA MILK, CEREAL MILK (초코 우유, 바나나 우유, 시리얼 우유)

    *** 기존 초코 우유 품목에서, 우유와 함께 자주 구매되는 상품인 <U>시리얼과 바나나</U>를 새로운 맛으로 활용하여 제품을 출시!!

🧊 운영 방법 :

- 우유 냉장고에 같은 브랜드를 기준으로 진열하여, 이미 흰 우유를 소비하고 있는 소비자들을 대상으로 해당 제품 제조사의 다른 우유 제품을 맛보기 쉽게 구매 유도

- 골라먹는 이벤트를 진행, 2가지 우유 상품을 함께 구매하면 할인 혜택을 제공

- 아이들의 관심을 끌기 위해, 우유팩의 포장지 색깔도 해당 맛을 떠올리기 쉽게 알록달록한 패키징 사용

- 새로운 맛을 출시했기 때문에, 시음 행사도 진행

📈 기대 효과 :

- 우유 상품 구매율 상승

- 새로 출시된 상품의 구매 증가율을 분석하여, 새로운 맛을 출시하며 지속적인 매출 상승 기대 가능

- 특정 우유 제조사의 제품 매출 상승 시, 잉마트와 해당 기업이 함께 프로모션을 진행, 쿠폰 지급 및 할인 혜택 적용 등을 강화하여 소비자들의 소비 촉진 효과 발생

- 소비자들이 우유 냉장고에 자주 노출되면서, 타 유제품 품목들에도 자연스레 더 자주 노출되어 구매 유도 가능

- 전략 2:

### [🍞국+밥이 있다면, 빵+우유가 있다🥐]

👨‍🍳 목적 :
* 베이커리 관련 품목과 우유 품목을 함께 구매하도록 유도

🍞 구성 품목 :

* MAINSTREAM WHITE BREAD 중심의 베이커리 품목, 우유 품목

    \+ 우유 상품과 베이커리 상품이 결합된 상품 출시 (ex. A 우유를 사용한 빵 판매)

🥯 운영 방법 :

* 우유 냉장고 근처에서 빵 시식 행사를 진행하여 소비자들에게 우유와 빵 상품이 잘 어울린다는 것을 어필

* 이벤트 코너를 운영하여, 빵 빨리 먹기 대회 진행 -> 참가자들이 우유를 빠르게 찾는 모습을 통해 참가자 본인을 포함하여 다른 소비자들도 해당 빵이나 우유 품목에 관심을 갖도록 유도

    \+ 기존 우유 품목을 사용한 새로운 빵 제품이 출시되어 더 부드럽게 즐길 수 있다는 점을 어필하여 판매

📈 기대 효과 :

* 우유와 베이커리류 상품 구매율 상승

    \+ 우유와 빵을 결합한 상품을 통해, 두 가지 상품을 같이 구매하던 소비자들을 포함하여 소비자들에게 구매 유도 가능



# **🤓 기가 막힌 전략을 제시하는 분께는 행운이 찾아옵니다~🍀**